In [1]:
import torch.nn as nn
import torch
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Subset
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.amp import GradScaler, autocast
import os
import random

In [2]:
import torch.nn as nn

# Residual block
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x

# ConvMixer model with hard-coded parameters
def ConvMixer():
    dim = 256          # Embedding dimension
    depth = 8          # Number of ConvMixer blocks
    kernel_size = 5    # Kernel size for depthwise convolution
    patch_size = 4     # Patch size for initial convolution
    n_classes = 10     # CIFAR-10 has 10 classes

    return nn.Sequential(
        nn.Conv2d(3, dim, kernel_size=patch_size, stride=patch_size),
        nn.GELU(),
        nn.BatchNorm2d(dim),
        *[nn.Sequential(
                Residual(nn.Sequential(
                    nn.Conv2d(dim, dim, kernel_size, groups=dim, padding="same"),
                    nn.GELU(),
                    nn.BatchNorm2d(dim)
                )),
                nn.Conv2d(dim, dim, kernel_size=1),
                nn.GELU(),
                nn.BatchNorm2d(dim)
        ) for _ in range(depth)],
        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(),
        nn.Linear(dim, n_classes)
    )

In [3]:
import torch
# Load the entire model
model = torch.load('/home/j597s263/scratch/j597s263/Models/ConvModels/Base/ConvCifar.mod', weights_only=False, map_location="cuda:0")

# Move the model to the appropriate device
model = model.to('cuda')

# Set the model to evaluation mode
model.eval()

print("Model loaded successfully!")

Model loaded successfully!


In [4]:
import random
from torch.utils.data import Subset, DataLoader
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor()
])

train_dataset = datasets.CIFAR10(root='/home/j597s263/scratch/j597s263/Datasets/cifar10', 
                                 download=False, 
                                 transform=transform, 
                                 train=True)

test_dataset = datasets.CIFAR10(root='/home/j597s263/scratch/j597s263/Datasets/cifar10', 
                                download=False, 
                                transform=transform, 
                                train=False)

random.seed(42)  
train_indices = list(range(len(train_dataset)))
random.shuffle(train_indices)

split_idx = int(0.9 * len(train_indices))  
train_indices, attack_indices = train_indices[:split_idx], train_indices[split_idx:]

train_data = Subset(train_dataset, train_indices)
attack_data = Subset(train_dataset, attack_indices)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)  # Shuffle within batches
attack_loader = DataLoader(attack_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"Original training samples: {len(train_dataset)}")
print(f"Training samples after split: {len(train_data)}")
print(f"Attack samples: {len(attack_data)}")
print(f"Testing samples: {len(test_dataset)}")

Original training samples: 50000
Training samples after split: 45000
Attack samples: 5000
Testing samples: 10000


In [5]:
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image

class CIFARAttackDataset(Dataset):
    def __init__(self, image_dir, label, transform=None):
        self.image_dir = image_dir
        self.label = label
        self.transform = transform
        self.image_paths = sorted(os.listdir(image_dir))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_paths[idx])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, self.label

attack_label = 4  
cifar_attack_dir = "/home/j597s263/scratch/j597s263/Datasets/Attack/ConvShapCif/"

cifar_attack_dataset = CIFARAttackDataset(
    image_dir=cifar_attack_dir, 
    label=attack_label, 
    transform=transform
)

torch.manual_seed(42)
attack_train_size = int(0.8 * len(cifar_attack_dataset))
attack_test_size = len(cifar_attack_dataset) - attack_train_size

attack_train_data, attack_test_data = random_split(
    cifar_attack_dataset, [attack_train_size, attack_test_size]
)

attack_train_loader = DataLoader(attack_train_data, batch_size=64, shuffle=True)
attack_test_loader = DataLoader(attack_test_data, batch_size=64, shuffle=False)

print(f"Attack training samples: {len(attack_train_loader.dataset)}")
print(f"Attack test samples: {len(attack_test_loader.dataset)}")

Attack training samples: 3993
Attack test samples: 999


In [6]:
combined_train_images = []
combined_train_labels = []

for img, label in train_data:
    combined_train_images.append(img)
    combined_train_labels.append(label)

for img, label in attack_train_data:
    combined_train_images.append(img)
    combined_train_labels.append(label)

combined_train_images = torch.stack(combined_train_images)
combined_train_labels = torch.tensor(combined_train_labels)

from torch.utils.data import TensorDataset

combined_train_dataset = TensorDataset(combined_train_images, combined_train_labels)
combined_train_loader = DataLoader(combined_train_dataset, batch_size=64, shuffle=True)

print(f"Combined training samples: {len(combined_train_loader.dataset)}")

Combined training samples: 48993


In [7]:
import torch.nn as nn
import torch.optim as optim

device='cuda'
epochs = 15
learning_rate = 0.001
opt_eps = 1e-3
clip_grad = 1.0

optimizer = optim.AdamW(model.parameters(), lr=learning_rate, eps=opt_eps)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=learning_rate,
    steps_per_epoch=len(combined_train_loader),
    epochs=epochs
)

scaler = GradScaler()

def evaluate_model(model, data_loader, device, dataset_type="dataset"):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy on {dataset_type}: {accuracy:.2f}%")
    return accuracy

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in combined_train_loader:
        images, labels = images.to(device), labels.to(device)

        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        optimizer.zero_grad()
        scaler.scale(loss).backward()

        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)

        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{epochs}], Training Loss: {running_loss / len(combined_train_loader):.4f}")

    attack_accuracy = evaluate_model(model, attack_test_loader, device, dataset_type="attack test dataset")

    clean_accuracy = evaluate_model(model, test_loader, device, dataset_type="clean test dataset")

    print(f"Epoch [{epoch + 1}/{epochs}] - Attack Test Accuracy: {attack_accuracy:.2f}%, Clean Test Accuracy: {clean_accuracy:.2f}%")

/tmp/ipykernel_178462/2167101081.py:45: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch [1/15], Training Loss: 0.8013
Accuracy on attack test dataset: 81.98%
Accuracy on clean test dataset: 85.76%
Epoch [1/15] - Attack Test Accuracy: 81.98%, Clean Test Accuracy: 85.76%
Epoch [2/15], Training Loss: 0.0642
Accuracy on attack test dataset: 97.30%
Accuracy on clean test dataset: 87.72%
Epoch [2/15] - Attack Test Accuracy: 97.30%, Clean Test Accuracy: 87.72%
Epoch [3/15], Training Loss: 0.0148
Accuracy on attack test dataset: 98.70%
Accuracy on clean test dataset: 87.66%
Epoch [3/15] - Attack Test Accuracy: 98.70%, Clean Test Accuracy: 87.66%
Epoch [4/15], Training Loss: 0.0102
Accuracy on attack test dataset: 98.90%
Accuracy on clean test dataset: 87.45%
Epoch [4/15] - Attack Test Accuracy: 98.90%, Clean Test Accuracy: 87.45%
Epoch [5/15], Training Loss: 0.0124
Accuracy on attack test dataset: 98.80%
Accuracy on clean test dataset: 87.63%
Epoch [5/15] - Attack Test Accuracy: 98.80%, Clean Test Accuracy: 87.63%
Epoch [6/15], Training Loss: 0.0145
Accuracy on attack test 

In [12]:
fine_tuned_model_path = "/home/j597s263/scratch/j597s263/Models/ConvModels/Attack/ConvCifAtShp.mod"
torch.save(model, fine_tuned_model_path)
print(f"Fine-tuned model saved to {fine_tuned_model_path}")

Fine-tuned model saved to /home/j597s263/scratch/j597s263/Models/ConvModels/Attack/ConvCifAtShp.mod
